In [1]:
from fabric.api import local
import json, os

# Return str
def ReadFile(path):
  with open(path, 'r') as f:
    output = f.read().replace("'","\"")
  return output

# Return specific commit message
def CommitMessage(oauth_token, github_api_url, owner, repo, commit_id):
    commit_raw = local('curl -H "Authorization: token {0}" {1}/repos/{2}/{3}/commits/{4}'.format(oauth_token, github_api_url, owner, repo, commit_id), capture=True)
    message = json.loads(str(commit_raw))["commit"]["message"]
    print message
    return message

# Return last commit's message
def CurrentCommitMessage(oauth_token, github_api_url, owner, repo, git_current_branch, git_current_commit_id):
    commit_raw = local('curl -H "Authorization: token {0}" {1}/repos/{2}/{3}/commits/{4}'.format(oauth_token, github_api_url, owner, repo, git_current_commit_id), capture=True)
    git_head_commit_id = json.loads(str(commit_raw))['parents'][-1]['sha']
    print "git_head_commit_id: " + git_head_commit_id
    return CommitMessage(oauth_token, github_api_url, owner, repo, git_head_commit_id)

# post comment to github
def GithubCreateComment(oauth_token, github_api_url, owner, repo, git_current_branch, path_log):
    
    # read upper stream build log
    text = ReadFile(path_log)
    
    # extract issue number from build log
    issue_num = text.split(" ")[3].replace("#", "")
    
    # dummy message
    greeting = "Hello Guys. I am Mr.Robot. \n"

    # json text
    comment = {'body': greeting + "```\n" + text + "```\n"}

    # posting comment
    local('curl -H \"Authorization: token {0}\" {1}/repos/{2}/{3}/issues/{4}/comments -X POST -d \' {5}\''.format(oauth_token, github_api_url, owner, repo, issue_num, json.dumps(comment)))

In [ ]:
CurrentCommitMessage:
oauth_token=4afa05f64a196fc7981e8fc44beca5c7a73dd513,
github_api_url=https://api.github.com,owner=boyd-mask,repo=git-jenkins-test,
git_current_branch=origin/pr/4/merge,
git_current_commit_id=228061cfd23a38e7a6453fa8c6eabf222e17940f

In [7]:
# default variables
oauth_token="4afa05f64a196fc7981e8fc44beca5c7a73dd513"
github_api_url="https://api.github.com"

owner = 'boyd-mask'
repo = 'git-jenkins-test'

gir_url ="https://github.com/boyd-mask/git-jenkins-test/"
git_current_commit_id = "228061cfd23a38e7a6453fa8c6eabf222e17940f"
git_current_branch = "origin/master"

path_log="/usr/local/opt/jenkins/jobs/ghpqb-sql-test/builds/34/log"
path_log=os.getcwd() + '/log'

In [78]:
# GithubCreateComment(oauth_token,github_api_url,owner,repo,git_current_branch,path_log)

In [79]:
# /usr/local/bin/fab -f /usr/local/jenkins/fabfile.py 
# GithubCreateComment:
# oauth_token=${OAUTH_TOKEN},
# github_api_url=${GITHUB_API_URL},
# owner=${OWNER},
# repo=${REPO},
# git_current_branch=${GIT_BRANCH},
# path_log=/usr/local/opt/jenkins/jobs/${UpperJobName}/builds/${UpperBuildNumber}/log

In [ ]:
# /usr/local/bin/fab -f /usr/local/jenkins/fabfile.py 
# CurrentCommitMessage:
# oauth_token=${OAUTH_TOKEN},
# github_api_url=${GITHUB_API_URL},
# owner=${OWNER},
# repo=${REPO},
# git_current_branch=${GIT_BRANCH},
# git_current_commit_id=${GIT_COMMIT}